In [35]:
from pygraylog.graylogapi import GraylogAPI
import json
import sys
import numpy as np
import pandas as pd

In [36]:
log_level = 0

In [37]:
graylog_con={
        "user":"merger",
        "password":"pc@pM8rger",
        "url":"http://graylog-ui.roamability.com:9000/api"
    }

In [38]:
api = GraylogAPI(
    graylog_con["url"],
    graylog_con["user"], 
    graylog_con["password"])

In [39]:
import datetime
#format like 2017-03-06T16:28
dateFrom = datetime.datetime.strptime('2018-01-12T05:00', '%Y-%m-%dT%H:%M')
dateTo = datetime.datetime.strptime('2018-01-12T12:00', '%Y-%m-%dT%H:%M')
query='protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:"a2:2a:0c:40"'

In [40]:
gres=api.search.universal.absolute.get(query=query, from_=dateFrom, to=dateTo)

In [41]:
gres

u'{"query":"protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:\\"a2:2a:0c:40\\"","built_query":"{\\"from\\":0,\\"size\\":150,\\"query\\":{\\"bool\\":{\\"must\\":{\\"query_string\\":{\\"query\\":\\"protocol:ss7 AND opCode_localValue:2 AND TCAP_type:Begin AND e212_imsi:425191201068102 AND tcap_otid:\\\\\\"a2:2a:0c:40\\\\\\"\\",\\"allow_leading_wildcard\\":false}},\\"filter\\":{\\"bool\\":{\\"must\\":{\\"range\\":{\\"timestamp\\":{\\"from\\":\\"2018-01-12 04:00:00.000\\",\\"to\\":\\"2018-01-12 11:00:00.000\\",\\"include_lower\\":true,\\"include_upper\\":true}}}}}}},\\"sort\\":[{\\"timestamp\\":{\\"order\\":\\"desc\\"}}]}","used_indices":[{"index_name":"graylog_41","begin":"1970-01-01T00:00:00.000Z","end":"1970-01-01T00:00:00.000Z","calculated_at":"2018-01-22T18:31:43.745Z","took_ms":0},{"index_name":"graylog_36","begin":"2018-01-11T20:58:35.461Z","end":"2018-01-14T00:41:28.217Z","calculated_at":"2018-01-14T00:43:34.379Z","took_ms":4767}],

In [42]:
def outLog(message, level=1):
    if(level <= log_level):
        print (message)
        sys.stdout.flush()

In [43]:
def LoadGraylogResult(exportsearch_results):
        jsonResult = json.loads(exportsearch_results)
        outputResult = list()
        i=0
        if "total_results" in jsonResult:
            outLog("Graylog search found {} messages".format(jsonResult["total_results"]),0)

        # Get the results and store them
            for result in jsonResult["messages"]:  
                if "message" in  result and isinstance(result["message"], dict):
                    outputResult.append(result["message"])
                    i+=1
                    if i%100==0: outLog("loaded {} records from Graylog".format(i),0)
            outLog("Graylog search load {} messages".format(len(outputResult)),0)
        else:
            outLog("Graylog search error: {}".format(exportsearch_results),0)

        return outputResult

In [44]:
pgres=LoadGraylogResult(gres)

Graylog search found 1 messages
Graylog search load 1 messages


In [45]:
pgres

[{u'TCAP_type': u'Begin',
  u'_id': u'd39e7de2-f760-11e7-8ae9-00505698f35d',
  u'collector_node_id': u'graylog-collector-sidecar',
  u'e164_msisdn': u'972559900040',
  u'e212_imsi': u'425191201068102',
  u'e212_mcc': u'425',
  u'e212_mnc': u'19',
  u'facility': u'filebeat',
  u'file': u'/mnt/tracing2ch/sigtran-public/sigtran-public_06090_20180112070835.pcap',
  u'frame_number': u'3369',
  u'frame_time_epoch': u'1515737359.744715747',
  u'gl2_remote_ip': u'172.19.11.192',
  u'gl2_remote_port': 46264,
  u'gl2_source_collector': u'b3d2019c-476e-439e-8764-043b24bcda9d',
  u'gl2_source_input': u'5a12819aab324f51d795a633',
  u'gl2_source_node': u'be115c61-9b16-42d7-810c-8f0e4af6a69c',
  u'gsm_map_ms_supportedCamelPhases': u'c0',
  u'gsm_map_nature_of_number': u'1',
  u'input_type': u'log',
  u'm3ua_DPC': u'7138',
  u'm3ua_OPC': u'701',
  u'message': u'12/01/2018 07:09:19.744716 \tprotocol=ss7\tfile=/mnt/tracing2ch/sigtran-public/sigtran-public_06090_20180112070835.pcap\tframe_time_epoch=1515

In [50]:
data=pgres[0]

In [52]:
data['e212_imsi']

u'425191201068102'

In [53]:
columns=list(data.keys())
values = list(data.values())
arr_len = len(values)
pd.DataFrame(np.array(values, dtype=object).reshape(1, arr_len), columns=columns)

,sccp_class,e164_msisdn,protocol,gl2_source_collector,offset,opCode_localValue,e212_mnc,sccp_message_type,m3ua_DPC,file,...,gl2_source_node,m3ua_OPC,sccp_called_digits,gsm_map_ms_supportedCamelPhases,tcap_otid,e212_mcc,streams,sccp_calling_ri,_id,frame_number
0,1,972559900040,ss7,b3d2019c-476e-439e-8764-043b24bcda9d,911662,2,19,0x00000009,7138,/mnt/tracing2ch/sigtran-public/sigtran-public_...,...,be115c61-9b16-42d7-810c-8f0e4af6a69c,701,972559912010681,c0,a2:2a:0c:40,425,[000000000000000000000001],0,d39e7de2-f760-11e7-8ae9-00505698f35d,3369
